In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, ToolMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict
from typing import Annotated, List
import json
from dotenv import load_dotenv
import os

# LangSmith 통합을 위한 임포트 추가
from langsmith import traceable
from langchain.callbacks.tracers.langchain import wait_for_all_tracers
from langsmith.run_trees import RunTree

# 환경 변수 로드
load_dotenv()



openai_model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

class State(TypedDict):
    messages: Annotated[list, add_messages]

# Tavily 검색 도구 설정
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 초기화 시 traceable로 래핑
@traceable(name="tavily_search")
def search_tavily(query):
    tool = TavilySearchResults(max_results=2)
    return tool.invoke(query)

tool = TavilySearchResults(max_results=2)
print(f"Tool name: {tool.name}")
print(f"Tool description: {tool.description}")
print(f"Tool args: {tool.args}")

# LLM 설정 및 도구 바인딩
llm = ChatOpenAI(model=openai_model)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

# 추적 가능한 챗봇 함수
@traceable(name="chatbot_node")
def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# 도구 노드 클래스 및 함수
class ToolNode:
    def __init__(self, tools):
        self.tools_by_name = {tool.name: tool for tool in tools}

    # 추적 가능한 도구 호출 함수
    @traceable(name="tool_execution")
    def execute_tool(self, tool_name, tool_args, tool_call_id):
        tool_result = self.tools_by_name[tool_name].invoke(tool_args)
        return tool_result

    def __call__(self, inputs):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No messages found in inputs.")
        
        outputs = []
        for tool_call in message.tool_calls:
            # 추적 가능한 함수를 통해 도구 실행
            tool_result = self.execute_tool(
                tool_call['name'], 
                tool_call['args'],
                tool_call['id']
            )
            
            tool_message = ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call['name'],
                tool_call_id=tool_call['id'],
            )
            outputs.append(tool_message)

        return {"messages": outputs}
    
tool_node = ToolNode(tools)

# 도구 조건 함수
@traceable(name="tools_routing")
def tools_condition(state):
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return 'tools'
    return END

# LangGraph 워크플로우 설정
@traceable(name="build_workflow")
def create_workflow():
    workflow = StateGraph(State)
    
    workflow.add_node("chatbot", chatbot)
    workflow.add_node("tools", tool_node)
    
    workflow.add_edge(START, "chatbot")
    workflow.add_conditional_edges("chatbot", tools_condition, {"tools": "tools", END: END})
    workflow.add_edge("tools", "chatbot")
    
    # LangSmith 트레이스가 포함된 컴파일된 그래프 반환
    return workflow.compile()

# 워크플로우 생성
graph = create_workflow()

# 시각화 (IPython 환경에서 사용)
# from IPython.display import Image, display
# display(Image(graph.get_graph().draw_mermaid_png()))

# 추적 가능한 전체 실행 함수
@traceable(name="main_conversation", inputs_to_trace=["user_input"])
def run_conversation(user_input):
    state = {"messages": [HumanMessage(content=user_input)]}
    response = graph.invoke(state)
    return response

# 사용 예시
if __name__ == "__main__":
    user_input = "LangGraph에서 '노드'란 무엇인가요?"
    response = run_conversation(user_input)
    print(response["messages"][-1].content)
    
    # 모든 트레이스가 완료될 때까지 대기 (비동기 전송을 위해)
    wait_for_all_tracers()

Tool name: tavily_search_results_json
Tool description: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
Tool args: {'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}
LangGraph에서 '노드'는 주로 데이터의 구조적 구성 요소를 의미하며, 일반적으로 정보의 단위 또는 요소로 사용됩니다. LangGraph는 프로그램의 구성 요소를 시각적으로 나타내고, 서로 연결된 노드들로 구성된 그래프 형태로 데이터를 표현합니다.

1. [Brunch](https://brunch.co.kr/@@hqFh/132)에서 LangGraph는 데이터의 관계를 Python 형태로 표현하는 도구로 설명되며, 노드는 해당 데이터의 기본 단위를 형성합니다.
2. [Kyobo Blog](https://blog.kyobodts.co.kr/2025/02/28/ai-agent-%EA%B0%9C%EB%B0%9C%EC%9D%98-%ED%95%B5%EC%8B%AC-rag%EC%99%80-langchain-langgraph-%EB%B9%84%EA%B5%90-%EB%B6%84%EC%84%9D/)에서는 LangGraph가 객체 지향적 패턴을 통해 노드 간의 관계를 정리하는 방법에 대해 설명합니다.

따라서, LangGraph에서 '노드'는 각기 다른 데이터 요소를 연결하여 시각화하고, 이들 사이의 관계를 정의하는 역할을 합니다.
